## Import Necessary Dependencies


In [45]:
# OS
import os

# Time
import time

#CSV 
import csv

# Folder Path Setup
from decouple import config
USERNAME = config('USERNAME')
KEY = config('KEY')

# Excel Sheets
import csv

# ###########
#   Modules - Classes that contain functions used within code
# #############
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

# Beautiful Soup
from bs4 import BeautifulSoup
# # Login Module ( For Using Login Class )
# from modules.login import Login as login
# # Go To Homepage Module ( Making Company Search Query and Navigating to Homepage )
# from modules.go_to_homepage import GoToHomepage
# # Get Search Module 
# from modules.get_search import GetSearch
# # Get Search Module 
# from modules.collect import Collect

driver = Service('driver/chromedriver')
browser = webdriver.Chrome(service=driver)

## Add Companies and Queries (Search Terms)
 

In [46]:
# GET COMPANIES
companies = []
file_path = "companies.txt"
with open(file_path, 'r') as companies_list:
    for i in companies_list.readlines():
        name= i.replace('\n', "")
        companies.append(name)


# SEARCH TERMS
search_terms = ['CEO', 'Chief Revenue Officer', 'Chief Operating Officer']

## Set headers for Exported CSV

In [47]:
# Set Excel Header Row
headers = ["Company", "CEO", "CRO", "COO"]

with open('final_list.csv', 'a') as f:
    object = csv.writer(f)
    object.writerow(headers)
    f.close()

## Login

In [48]:
# Login
def login(username, key):
    browser.get('https://www.linkedin.com/uas/login')
    elementID = browser.find_element(By.ID, 'username')
    elementID.send_keys(username)
    elementID = browser.find_element(By.ID, 'password')
    elementID.send_keys(key)
    elementID.submit()   

login(USERNAME, KEY)

## Search Function

In [49]:
class Search():

    def __init__(self, browser, search_terms, time):
        self.browser = browser
        self.search_terms = search_terms
        self.time = time


    # Go To Homepage Method
    def go_to_page(self, company):
        #  Every company search on linkedin will contain this url syntax or prefix
        linkedin_search_baseurl = "https://www.linkedin.com/search/results/companies/?keywords="

        # linkedin url's have a dynamic syntax, therefore in order to search for companies whose name contains more than one word, we will need to concatentate the individual words with linkedin's uniuqe syntax ('%20' is placed between each word)

        # split the company name so each word is given an index
        company_word_list = company.split()

        # we need to create a variable that will hold the dynamic suffix to our base linkedin company search url.
        url_ending = ""

        # if the company name is just one word, then the url_ending will simply be the name of the company
        if len(company_word_list) == 1:
            url_ending += company_word_list[0]

        # if the company name is longer than 1 word, the search url syntax will change - For this, we can run a loop over each word in the company name, and if the current word is not first, place '$20' before the word at that current word's index.
        else:
            url_ending = company_word_list[0]
            i = 1
            while i < len(company_word_list):
                # each time the loop is ran, a value with be added to url_ending variable we've created
                url_ending += "%20" + company_word_list[i]
                i += 1

        # Company Insights URL
        company_url = linkedin_search_baseurl + url_ending
        # Now that we've identified what the url ending should be for the unique company we're researching, we can now navigate to the corresponding LI search query results

        self.browser.get(company_url)
        self.browser.implicitly_wait(2)
        self.time.sleep(4)

        # Company Linkedin Page URL
        company_li_url = self.browser.find_element(By.XPATH,'//div[@class="entity-result__item"]//a').get_attribute('href')
        self.browser.get(company_li_url)
        self.time.sleep(4)

        data = []
        for search_term in self.search_terms:
            results = self.get_search(company, search_term, company_li_url)
            data.append(results)

        ceo=data[0]
        print(ceo)
        cro=data[1]
        print(cro)
        coo=data[2]
        print(coo)



        row = [company, ceo, cro, coo]

        with open("final_list.csv", 'a') as csvfile: 
            object = csv.writer(csvfile)
            object.writerow(row)
            csvfile.close()

    def get_search(self, company, search_query, company_li_url):
        self.time.sleep(4) 
        #Navigate to company homepage by clicking on search query link. 
        #Use direct selenium to grab company employee search query page 
        company_search_query_page_parent_element = browser.find_element(
            By.CLASS_NAME, 'display-flex.mt2.mb1')
        company_search_query_page = company_search_query_page_parent_element.find_element(By.CLASS_NAME,
        'ember-view')
        company_search_query_page.click()
        self.time.sleep(4)
        search_query_url = str(self.browser.current_url)
        position = search_query_url.find('COMPANY_PAGE')
        base_emp_search_url = search_query_url[:position]
        route = f'{base_emp_search_url}FACETED_SEARCH&title={search_query}'
        self.browser.get(route)
        self.time.sleep(4)
        employees = self.collect(company, search_query, company_li_url)
        return employees
        

    def collect(self, company, search_query, company_li_url):
        self.time.sleep(4)
        try:
            # Load page content using beautiful soup
            src = browser.page_source
            soup = BeautifulSoup(src)
            ul_tag = soup.find(
                'ul', {'class': 'reusable-search__entity-result-list list-style-none'})
            li_tags = ul_tag.find_all('li')
            # Loop over each list item and collect data
            employees = []
            for li_tag in li_tags:
                # [Company, Search Query, Name, Title, Location, Employee ]
                # Find name and profile link ( Same element )
                name_parent = li_tag.find(
                    'span', {'class', 'entity-result__title-text t-16'})
                target_element = name_parent.find("a", href=True)
                name = target_element.get_text().strip()
                page_link = target_element['href']
                
                # Find title
                title = li_tag.find(
                    'div', {'class': 'entity-result__primary-subtitle t-14 t-black t-normal'}).get_text().strip()

                # Find location
                location = li_tag.find(
                    'div', {'class': 'entity-result__secondary-subtitle t-14 t-normal'}).get_text().strip()
                
                employee = {
                    name: name, 
                    title: title, 
                    page_link: page_link, 
                    location: location
                }
                employees.append(employee)

            # print(employees)
            browser.get(company_li_url)
            self.time.sleep(2)            
            return employees

            

        except:
            employees = []
            self.time.sleep(2)
            return employees

        # Navigate back to company homepage 


#Login
search = Search(browser, search_terms, time)
for company in companies: 
    search.go_to_page(company)
    
# browser.close()

[{'LinkedIn Member': 'LinkedIn Member', 'CEO / Co-founder of Rael': 'CEO / Co-founder of Rael', 'https://www.linkedin.com/search/results/people/headless?currentCompany=%5B42842093%5D&title=CEO&origin=FACETED_SEARCH': 'https://www.linkedin.com/search/results/people/headless?currentCompany=%5B42842093%5D&title=CEO&origin=FACETED_SEARCH', 'Los Angeles, CA': 'Los Angeles, CA'}, {'LinkedIn Member': 'LinkedIn Member', 'KRAFTON Inc. Chief Executive Officer': 'KRAFTON Inc. Chief Executive Officer', 'https://www.linkedin.com/search/results/people/headless?currentCompany=%5B42842093%5D&title=CEO&origin=FACETED_SEARCH': 'https://www.linkedin.com/search/results/people/headless?currentCompany=%5B42842093%5D&title=CEO&origin=FACETED_SEARCH', 'South Korea': 'South Korea'}, {'Sean (Hyunil) SohnView Sean (Hyunil) Sohn’s profile': 'Sean (Hyunil) SohnView Sean (Hyunil) Sohn’s profile', 'CEO at PUBG India, Head of India Div. at Krafton Inc.': 'CEO at PUBG India, Head of India Div. at Krafton Inc.', 'https

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".display-flex.mt2.mb1"}
  (Session info: chrome=98.0.4758.102)
Stacktrace:
0   chromedriver                        0x0000000101d30e69 chromedriver + 5160553
1   chromedriver                        0x0000000101cbb593 chromedriver + 4679059
2   chromedriver                        0x000000010186f2c8 chromedriver + 172744
3   chromedriver                        0x00000001018a4b62 chromedriver + 392034
4   chromedriver                        0x00000001018a4d21 chromedriver + 392481
5   chromedriver                        0x00000001018d7304 chromedriver + 598788
6   chromedriver                        0x00000001018c23fd chromedriver + 513021
7   chromedriver                        0x00000001018d50ab chromedriver + 589995
8   chromedriver                        0x00000001018c2623 chromedriver + 513571
9   chromedriver                        0x0000000101897dce chromedriver + 339406
